In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import pdb
import numpy as np
import numpy.random as npr
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import re
from sklearn.decomposition import PCA
from numpy import linalg as LA
from scipy.spatial import distance
from scipy.linalg import subspace_angles

import getpass
ui = getpass.getuser()
if ui == 'laura':
    p = '/home/laura'
elif ui == 'lauradriscoll':
    p = '/Users/lauradriscoll/Documents'

net = 'stepnet'
PATH_YANGNET = os.path.join(p,'code/multitask-nets',net)
sys.path.insert(0, PATH_YANGNET)

from task import generate_trials, rule_name, rule_index_map, rules_dict
from network import Model
import tools

from tools_lnd import gen_trials_from_model_dir, gen_X_from_model_dir, gen_X_from_model_dir_epoch, name_best_ckpt, find_ckpts, get_model_params

/Users/lauradriscoll/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dir_specific_all = 'lowD/combos'
rule_list = ['delaydm1', 'delaydm2']
model_n = 1
model_dir_all = os.path.join(p,'data/rnn/multitask',dir_specific_all,'most',str(model_n))
model = Model(model_dir_all)
ckpt_n = name_best_ckpt(model_dir_all,'delaygo')#
ckpt_n_dir = os.path.join(model_dir_all,'ckpts/model.ckpt-' + str(int(ckpt_n)))

fldr = os.path.join('romo/',dir_specific_all,str(model_n))
if not os.path.exists(fldr):
    os.makedirs(fldr)

Variables being optimized:
<tf.Variable 'rnn/leaky_rnn_cell/kernel:0' shape=(281, 256) dtype=float32_ref>
<tf.Variable 'rnn/leaky_rnn_cell/bias:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'output/weights:0' shape=(256, 3) dtype=float32_ref>
<tf.Variable 'output/biases:0' shape=(3,) dtype=float32_ref>
Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [18]:
FONT_WEIGHT = 'bold'
n_pcs = 50
epoch = 'go1'
rule = 'delaydm1'
trial = gen_trials_from_model_dir(model_dir_all,rule)
epoch_set = ['fix1','stim1','delay1','stim2','delay2','go1']#trial.epochs.keys()
s = np.shape(trial.stim_locs)
e_ind = 1
n_pcs_plot = 4

fig = plt.figure(figsize=(24,24),tight_layout=True,facecolor='white')
plt.rcParams.update({'font.size': 13})
cmap=plt.get_cmap('rainbow')

for epoch in epoch_set:
    e_ind += 1

    trial = gen_trials_from_model_dir(model_dir_all,rule)
    X, x = gen_X_from_model_dir_epoch(model_dir_all,ckpt_n_dir,trial,epoch)
    N,S = np.shape(X) # neurons x trials (stim)
    s_list = np.tile(np.expand_dims(trial.stim_locs,axis=1),(1,int(S/s[0])))
    t_list = np.tile(np.expand_dims(range(int(S/s[0])),axis=1),s[0])
    row_means = np.mean(X, axis=1)
    X_center = X - row_means.reshape(X.shape[0],1)

    pcar = PCA(n_components=n_pcs)
    pcar.fit(X_center.T)
    X_pc = pcar.transform(X_center.T)
    
    for pc in range(n_pcs_plot):
        ax_all = fig.add_subplot(n_pcs_plot,len(epoch_set)+1,e_ind+(len(epoch_set)+1)*pc)
        for t in range(np.shape(trial.stim_locs)[0]):
            c = cmap(trial.stim_locs[t,0]/max(trial.stim_locs[:,0]))
            a = trial.stim_strength[t,0]/max(trial.stim_strength[:,0])
            ind = range(t*int(S/s[0]),(t+1)*int(S/s[0]))
            plt.plot(X_pc[ind,pc],c=c)
        plt.title(epoch)

    ax_all = fig.add_subplot(n_pcs_plot,len(epoch_set)+1,1)
    plt.plot(np.cumsum(pcar.explained_variance_)/np.sum(pcar.explained_variance_),'-o', markersize = 4, label = epoch)
    
plt.legend()
ax_all.set_ylim(0, 1.01)
ax_all.set_xlim(-1, 10)
ax_all.set_xlabel('Component #')
ax_all.set_ylabel('Fraction Variance Explained')
plt.show()


Variables being optimized:
<tf.Variable 'rnn/leaky_rnn_cell/kernel:0' shape=(281, 256) dtype=float32_ref>
<tf.Variable 'rnn/leaky_rnn_cell/bias:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'output/weights:0' shape=(256, 3) dtype=float32_ref>
<tf.Variable 'output/biases:0' shape=(3,) dtype=float32_ref>
INFO:tensorflow:Restoring parameters from /Users/lauradriscoll/Documents/data/rnn/multitask/lowD/combos/most/1/model.ckpt
Model restored from file: /Users/lauradriscoll/Documents/data/rnn/multitask/lowD/combos/most/1/model.ckpt
Variables being optimized:
<tf.Variable 'rnn/leaky_rnn_cell/kernel:0' shape=(281, 256) dtype=float32_ref>
<tf.Variable 'rnn/leaky_rnn_cell/bias:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'output/weights:0' shape=(256, 3) dtype=float32_ref>
<tf.Variable 'output/biases:0' shape=(3,) dtype=float32_ref>
INFO:tensorflow:Restoring parameters from /Users/lauradriscoll/Documents/data/rnn/multitask/lowD/combos/most/1/model.ckpt
Model restored from file: /Users/laura

ValueError: Invalid RGBA argument: array([[0.50392157, 0.99998103, 0.70492555, 1.        ]])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [19]:
fig = plt.figure(figsize=(24,8),tight_layout=True,facecolor='white')
plt.rcParams.update({'font.size': 13})

e_ind = 0
for epoch in epoch_set:
    e_ind += 1

    trial = gen_trials_from_model_dir(model_dir_all,rule)
    X, x = gen_X_from_model_dir_epoch(model_dir_all,ckpt_n_dir,trial,epoch)

    row_means = np.mean(X, axis=1)
    X_center = X - row_means.reshape(X.shape[0],1)

    pcar = PCA(n_components=n_pcs)
    pcar.fit(X_center.T)
    X_pc = pcar.transform(X_center.T)
    
#     for pc in range(n_pcs_plot):
    ax_all = fig.add_subplot(1,len(epoch_set)+1,e_ind+1)
    for t in range(np.shape(trial.stim_locs)[0]):
        c = cmap(trial.stim_locs[t,0]/max(trial.stim_locs[:,0]))
        a = trial.stim_strength[t,0]/max(trial.stim_strength[:,0])
        ind = range(t*int(S/s[0]),(t+1)*int(S/s[0]))
        plt.plot(X_pc[ind,0],X_pc[ind,1],'-',c=c)
    plt.title(epoch)
    
    ax_all = fig.add_subplot(1,len(epoch_set)+1,1)
    plt.plot(np.cumsum(pcar.explained_variance_)/np.sum(pcar.explained_variance_),'-o', markersize = 4, label = epoch)
    
    
plt.legend()
ax_all.set_ylim(0, 1.01)
ax_all.set_xlim(-1, 20)
ax_all.set_xlabel('Component #')
ax_all.set_ylabel('Fraction Variance Explained')
plt.show()

Variables being optimized:
<tf.Variable 'rnn/leaky_rnn_cell/kernel:0' shape=(281, 256) dtype=float32_ref>
<tf.Variable 'rnn/leaky_rnn_cell/bias:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'output/weights:0' shape=(256, 3) dtype=float32_ref>
<tf.Variable 'output/biases:0' shape=(3,) dtype=float32_ref>
INFO:tensorflow:Restoring parameters from /Users/lauradriscoll/Documents/data/rnn/multitask/lowD/combos/most/1/model.ckpt
Model restored from file: /Users/lauradriscoll/Documents/data/rnn/multitask/lowD/combos/most/1/model.ckpt
Variables being optimized:
<tf.Variable 'rnn/leaky_rnn_cell/kernel:0' shape=(281, 256) dtype=float32_ref>
<tf.Variable 'rnn/leaky_rnn_cell/bias:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'output/weights:0' shape=(256, 3) dtype=float32_ref>
<tf.Variable 'output/biases:0' shape=(3,) dtype=float32_ref>
INFO:tensorflow:Restoring parameters from /Users/lauradriscoll/Documents/data/rnn/multitask/lowD/combos/most/1/ckpts/model.ckpt-310000
Variables being optimized

IndexError: index 10000 is out of bounds for axis 0 with size 10000

In [ ]:
FONT_WEIGHT = 'bold'
n_pcs = 50
epoch = 'go1'
rule = 'delaydm1'
trial = gen_trials_from_model_dir(model_dir_all,rule)
s = np.shape(trial.stim_locs)
e_ind = 1
n_pcs_plot = 4

fig = plt.figure(figsize=(12,12),tight_layout=True,facecolor='white')
plt.rcParams.update({'font.size': 13})
cmap=plt.get_cmap('rainbow')
epoch_set = ['stim1','stim2']
rule = 'delaydm1'
X_all, _ = gen_X_from_model_dir(model_dir_all,ckpt_n_dir,trial)
row_means = np.mean(X_all, axis=1)

e_ind = 0
for epoch1 in epoch_set:
    e_ind += 1

    trial = gen_trials_from_model_dir(model_dir_all, rule, mode ='random')
    X, x = gen_X_from_model_dir_epoch(model_dir_all,ckpt_n_dir,trial,epoch1)
    
    X_center = X - row_means.reshape(X.shape[0],1)
    pcar = PCA(n_components=n_pcs)
    pcar.fit(X_center.T)
    
#     for pc in range(n_pcs_plot):
    for epoch2 in epoch_set:
        plot_ind = epoch_set.index(epoch2)+1+epoch_set.index(epoch1)*len(epoch_set)
        ax_all = fig.add_subplot(len(epoch_set),len(epoch_set),plot_ind)
        X, x = gen_X_from_model_dir_epoch(model_dir_all,ckpt_n_dir,trial,epoch2)
        N,S = np.shape(X) # neurons x trials (stim)
        X_center = X - row_means.reshape(X.shape[0],1)
        X_pc = pcar.transform(X_center.T)
        
        for t in range(np.shape(trial.stim_locs)[0]):
            c = cmap(trial.stim_locs[t,0]/max(trial.stim_locs[:,0]))
            a = trial.stim_strength[t,0]/max(trial.stim_strength[:,0])
            ind = range(t*int(S/s[0]),(t+1)*int(S/s[0]))
            plt.plot(X_pc[ind,2],X_pc[ind,1],'-',c=c,alpha = a)
        plt.title(epoch1+' '+epoch2)
    
plt.show()